# Protein Preparation Tutorial

This notebook demonstrates how to prepare protein structures for molecular docking using the drugforge toolkit.

## Setup

First, let's import the necessary modules

In [ ]:
from drugforge.data.testing.test_resources import fetch_test_file
from drugforge.data.schema.complex import Complex
from drugforge.modeling.schema import PreppedComplex
from drugforge.modeling.protein_prep import ProteinPrepper

Next we setup the example data

In [ ]:
complex_pdb = Complex.from_pdb(
        fetch_test_file("structure_dir/Mpro-x0354_0A_bound.pdb"),
        target_kwargs={"target_name": "test"},
        ligand_kwargs={"compound_name": "test2"},
    )

Let's examine it

In [ ]:
complex_pdb.target

In [ ]:
complex_pdb.ligand.to_rdkit()

# Now we prep the protein using OpenEye

In [ ]:
# this is used by OpenEye to fill in any missing loops
loop_db = fetch_test_file("fragalysis-mpro_spruce.loop_db")
target = "SARS-CoV-2"

In [ ]:
# this will take ~30 seconds
prepper = ProteinPrepper(loop_db=loop_db)
pcs = prepper.prep([complex_pdb], use_dask=False)

In [ ]:
prepped_complex = pcs[0]

In [ ]:
prepped_complex

Since we used the OpenEye prepper, we'll get a result that is basically a wrapper around the OpenEye design unit object. We can get the DesignUnit directly with:

In [ ]:
du = prepped_complex.target.to_oedu()
print(du.HasReceptor())
print(du.HasLigand())

We can save the pdb, design unit, and ligand sdfs.

In [ ]:
prepped_complex.target.to_oedu_file("test_prepped_complex.oedu")

In [ ]:
prepped_complex.target.to_pdb_file("test_prepped_complex.pdb")

In [ ]:
prepped_complex.ligand.to_sdf("test_prepped_complex.sdf")

We can serialize this whole object as a json as well! 

In [ ]:
prepped_complex.to_json_file("test_prepped_complex.json")

In [ ]:
loaded = PreppedComplex.from_json_file("test_prepped_complex.json")

In [ ]:
loaded == prepped_complex

# Other Utilities

While the whole protein prep workflow is in `drugforge.workflows.prep_workflows`, the majority of the functionality is in `drugforge.modeling`. 

Currently we've just implemented the default OpenEye protein prepper (used above), and the LigandTransferProteinPrepper, which creates a new prepped complex by transfering the ligand from an existing complex to an apo structure. This was useful for us in particular for generating complexes with, i.e. MERS bound to Ligand A, when we only had SARS-CoV-2 bound to ligand A. 